# Convolutional Neural Networks

* The data for this notebook was extracted from S. Zheng, Y. Zhao, G. Li, R. Wang, and J. Ni [Npj Comput. Mater.](https://www.nature.com/articles/s41524-019-0223-y) **5**, 84 (2019).

* A total of 2,000 composition formulas are included in the data set. Each used for the generation of a $10 \times 10$ pixels atomic table, where each pixel represents the element composition.

# Overview

<img src="https://miro.medium.com/v2/resize:fit:640/format:webp/1*1VJDP6qDY9-ExTuQVEOlVg.gif" alt="Convolution operation" align="right" style="width:300px; float:center"/>

In this notebook we will learn how to implement convolution operations in neural networks. Their architecture tipically consists of a series of filtering layers followed by a multi-layer perceptron. The size for these filters determines the *receptive field* of the convolution operation used to detect different features in the data. Put simple, the receptive field is the portion of the input data used to compute the activation of a neuron in a *feature map*, that corresponds to the ouput of the convolutional layer. There are three hyperparameters that control the size for the output of a convolutional layer. These are related as follows

$$ O = \frac{I - F + 2\,P}{S} + 1$$

where $O$, is the output size (height/length), $I$ is the input size (height/length), $F$ is the filter size, $P$ is the padding, and $S$ is the stride. For example, our $10 \times 10$ input  and a $3 \times 3$ filter with stride of 1 and padding 0 results in a $8 \times 8$ output. With a stride of 2 and padding 1 we get an output of $5 \times 5$ instead.

# Libraries

In [ ]:
import torch

from pathlib          import Path
from torch.utils.data import DataLoader
#
### Import local libraries
#
from model import ConvolutionalNeuralNetwork

## 1. Prepare data

First, we need to load our datasets. We will use the `train.pth` that includes the atomic table for 1,600 materials compositions, and the `test.pth` as validation set with analogous data.

In [ ]:
#
### Define data files
#
imhere = Path.cwd()

nnfile = imhere/'network.pth'
#
### Load data
#
train_set = torch.load(imhere/'train.pth')
test_set  = torch.load(imhere/'test.pth')

## 2. Settings and hyperparameters

Our optimization algorithm is the ADAptive Moment estimation, [Adam](https://arxiv.org/pdf/1412.6980.pdf), that is based on stochastic gradient descent. We will need to define the **learning rate** and the **weight decay**. The learning rate is a hyperparameter that controls how much we are adjusting the weights of our network with respect to the loss gradient, whereas the weight decay is a regularization term that penalizes large weights

The number of **epochs** is the number of times the learning algorithm will work through the entire training dataset. One epoch means that each sample in the training dataset has had an opportunity to update the internal model parameters.

The **batch size** is a hyperparameter that defines the number of samples to work through before updating the internal model parameters. The batch size is a compromise between the speed and the quality of the learning. The larger the batch size, the faster the learning, but the less accurate the updates. The smaller the batch size, the slower the learning, but the more accurate the updates.

Since we are interested in learning the regression for a continuous variable, we will use the Mean Squared Error **loss function**.
$$
\mathrm{MSE} = \frac{1}{N} \sum_{i=1}^{N} (y_i - \hat{y}_i)^2
$$

where $N$ is the number of samples in the training set, $y_i$ is the reference value, and $\hat{y}_i$ is predicted value.

The training and testing data may be used in a loop function,

~~~
for X, y in training: print(f'X shape = {X.shape}, y shape = {y.shape}')
~~~

each loop will automatically pass a total of `batch_size` samples to the neural network.

In [ ]:
#
### Training parameters
#
learnig_rate = 1e-2
weight_decay = 1e-5

epochs       = 20
batch_size   = 200 # 1, 2, 4, 5, 8, 10, 16, 20, 25, 32, 40, 50, 64, 80, 100, 160, 200, 320, 400, 800, 1600
test_epoch   = 10
#
### Define neural network
#
network = ConvolutionalNeuralNetwork()
#
### Optimizer and Loss
#
optimizer = torch.optim.Adam(params=network.parameters(), lr=learnig_rate, weight_decay=weight_decay)
criterion = torch.nn.MSELoss(reduction='mean')
#
### Training and testing data
#
training = DataLoader(train_set, shuffle=True,  batch_size=batch_size)
testing  = DataLoader(test_set,  shuffle=False, batch_size=batch_size)

## 3. Training

We can now train our neural network for the total `epochs` we selected and testing it every `test_epoch` epochs.

Passing training data to the networks consists in five steps:

1. Set the gradients to zero, `optimizer.zero_grad()`.

2. Pass batch to the network, `output = network(batch)`.

3. Compute the loss, `loss = criterion(output, y)`.

4. Perform backward pass, `loss.backward()`.

5. Perform the optimization step, `optimizer.step()`.

Keep in mind that during testing you **DO NOT** want to update the gradients in your neural network. Otherwise you will leak testing information and your model will also learn from the testing set. To prevent this from happening, you need to use the `torch.no_grad()` context manager.  This will prevent the gradient from being updated. 

In [ ]:
#
### Training loop
#
for epoch in range(epochs):

    # your code here for the training set

    print(f'{epoch+1},train,{loss:.2f}')

    if (epoch+1)%test_epoch == 0:
        # your code here for the testing set

        print(f'{epoch+1},test,{loss:.2f}')
#
### Save model
#
torch.save(network.state_dict(), nnfile)